# Linear Regression

In [ ]:
import pyspark
import sys

In [ ]:
import pyspark.sql.functions as fn

In [ ]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [ ]:
# Check spark app name
spark.sparkContext.appName

In [ ]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)

In [ ]:
# print runtime versions
# Python version
sys.version

In [ ]:
# Spark version
spark.version

### Exploring Data

In [ ]:
# load iris.csv into Spark dataframe
df = spark.read.csv('data/lr_dataset.csv', header=True, inferSchema=True)

In [ ]:
#validate the size of data
df.count(), len(df.columns)

In [ ]:
# First 5 rows of Iris dataset
df.show(5)

In [ ]:
df.printSchema()

In [ ]:
df.describe().show()

In [ ]:
# check for correlation
df.select(fn.corr('var_1','output')).show()

### Feature Engineering

In [ ]:
# display all column names
df.columns

In [ ]:
# vectorize all numerical columns into a single feature column
feature_cols = df.columns[:-1]
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
features_df = assembler.transform(df)

In [ ]:
# validate the presence of dense vectors 
features_df.printSchema()

In [ ]:
# view the details of dense vector
features_df.select('features').show(5,False)

In [ ]:
# only select the features and label column
model_df = features_df.select(['features', 'output'])

In [ ]:
# Reading for machine learning
model_df.show(10,False)

In [ ]:
# size of model df
model_df.count(), len(model_df.columns)

### Split Data - Train & Test sets

In [ ]:
# use Logistic Regression to train on the training set
train_df, test_df = model_df.randomSplit([0.70, 0.30], seed=42)

In [ ]:
train_df.count(), len(train_df.columns)

In [ ]:
test_df.count(), len(test_df.columns)

### Build Linear Regression Model 

In [ ]:
# Build Linear Regression model 
lin_Reg=LinearRegression(labelCol='output')

In [ ]:
# fit the linear regression model on training data set 
lr_model=lin_Reg.fit(train_df)

In [ ]:
lr_model.intercept

In [ ]:
lr_model.coefficients

In [ ]:
training_predictions=lr_model.evaluate(train_df)

In [ ]:
training_predictions.meanSquaredError

In [ ]:
training_predictions.r2

### Evaluate Model

In [ ]:
# make predictions on test data 
test_results=lr_model.evaluate(test_df)

In [ ]:
# view the residual errors based on predictions 
test_results.residuals.show(10,False)

In [ ]:
# coefficient of determination value for model
test_results.r2

In [ ]:
# RMSE
test_results.rootMeanSquaredError

In [ ]:
# MSE
test_results.meanSquaredError